In [ ]:
#hdtb-msb-vis > div:nth-child(5) > a

import requests
from bs4 import BeautifulSoup
from selenium import webdriver as wd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
import os
import sys
import urllib.request
import datetime

word = "마드리드 맛집"
'''
# 번역부분---------
import requests

def kor2eng(query):
    url = "https://translate.kakao.com/translator/translate.json"

    headers = {
        "Referer": "https://translate.kakao.com/",
        "User-Agent": "Mozilla/5.0"
    }

    data = {
        "queryLanguage": "ko",
        "resultLanguage": "es",
        "q": query
    }

    resp = requests.post(url, headers=headers, data=data)
    data = resp.json()
    output = data['result']['output'][0][0]
    return output


if __name__ == "__main__":
#     print(kor2eng("コロナ19に注意してください。"))
    tword=kor2eng(word)
    print(kor2eng(word))
'''    
#----------------------------

def google_searching(word):
    url = "http://www.google.com/maps/search/" + word
    return url

from selenium import webdriver


driver = webdriver.Chrome('C:/work/chromedriver')

# 구글 접속하기
driver.get('http://www.google.com/maps')
time.sleep(3)
print(1)

#검색 후 목록 보기 
url = google_searching(word)
driver.get(url)
time.sleep(3)

results = []

import re
from bs4 import BeautifulSoup

for i in range(20):
    #목록에서 선택하기
    def select_first(driver):
        #pane > div > div.widget-pane-content.scrollable-y > div > div > div.section-layout.section-scrollbox.scrollable-y.scrollable-show.section-layout-flex-vertical > div.section-layout.section-scrollbox.scrollable-y.scrollable-show.section-layout-flex-vertical > div:nth-child(1) > div.section-result-content > div.section-result-text-content > div.section-result-header-container
        first = driver.find_elements_by_css_selector("div.section-result")[i]
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
    #         #first = soup.select('div.section-result-title-container > h3')[2]
        global title
        title = soup.select('div.section-result-title-container > h3')[0].text
        print(title)
        #print(first)
        first.click()
        time.sleep(3)
    select_first(driver)
    print('2')

    #리뷰로 넘어가기
    def select_second(driver):
        second = driver.find_element_by_css_selector("span button")
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
        #ugiz4pqJLAG__primary-text gm2-body-2
        global juso
        juso = driver.find_element_by_css_selector('div.ugiz4pqJLAG__text').text
        #juso = soup.select('div.ugiz4pqJLAG__text').text
        print(juso)
        second.click()
        time.sleep(3)
        #print(second)
    time.sleep(3)    
    select_second(driver)
    print('3')

    # 반복문 돌려서 리뷰 얻기.


    def get_content(driver):
        #  현재 페이지 html 정보 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')

        # 총 별점 가져오기
        tstar = soup.select('div.gm2-display-2')[0].text 
        print(tstar)

        t = 0
        s = 0
        e = 8
        while(t<20):
            time.sleep(5)
            if t == 0:
                s = 0
                e = 8
            else:
                s = 10 * (t-1) + 8
                e = s + 10
            for k in range(s, e):        
                #  리뷰 내용 가져오기
                try:
                    #time.sleep(1)
                    global content
                    #content = soup.select('div.section-review')[t].text
    #                 content = soup.select('div.section-review-review-content')[k].text
                    content = driver.find_element_by_css_selector('div.section-layout.section-scrollbox.scrollable-y.scrollable-show > div:nth-child(9) > div:nth-child({}) > div > div.section-review-content > div:nth-child(3) > div.section-review-review-content'.format(3 * k + 1)).text
    #                 print(content)

                    #  리뷰 내용이 길 경우 자세히 버튼 눌러서 가져오기
                    more = '자세히'
                    if more in content:
                        more.click()
                        content = driver.find_element_by_css_selector('div.section-layout.section-scrollbox.scrollable-y.scrollable-show > div:nth-child(9) > div:nth-child({}) > div > div.section-review-content > div:nth-child(3) > div.section-review-review-content'.format(3 * k + 1)).text
                        time.sleep(3)
    #                     print(content)

                except Exception as e:
                    print('err : ', e)
                    #content = ' '

                # 작성일 가져오기
                date = driver.find_element_by_css_selector('div.section-layout.section-scrollbox.scrollable-y.scrollable-show > div:nth-child(9) > div:nth-child({}) > div > div.section-review-content > div:nth-child(3) > div.section-review-metadata.section-review-metadata-with-note > span.section-review-publish-date'.format(3 * k + 1)).text
    #             date = soup.select('span.section-review-publish-date')[k].text
    #             print(date)

                # 리뷰에서 별점 가져오기
                try:
                    #star = driver.find_element_by_css_selector('span.section-review-stars')
                    global star
                    star = int(''.join(re.findall("\d+", driver.find_element_by_css_selector('div.section-layout.section-scrollbox.scrollable-y.scrollable-show > div:nth-child(9) > div:nth-child({}) > div > div.section-review-content > div:nth-child(3) > div.section-review-metadata.section-review-metadata-with-note > span.section-review-stars'.format(3 * k + 1))
                                                  .get_attribute('aria-label'))))
    #                 star = soup.select('span.section-review-stars')[k]['aria-label']
                    print(star)   
                except:
                    print('별점 에러')


                data = [title, juso, content, date, star]
    #             print(data)
                results.append(data)
            t += 1

            # 스크롤

            time.sleep(7)
            scrollable_div = driver.find_element_by_css_selector(
     'div.section-layout.section-scrollbox.scrollable-y.scrollable-show'
                         )
            driver.execute_script(
                   'arguments[0].scrollTop = arguments[0].scrollHeight', 
                    scrollable_div
                   )



    get_content(driver)
    # print(results)


import pandas as pd

results_df = pd.DataFrame(results)
results_df.columns = ['title', 'juso','content','date','star']
results_df.to_excel('C:\work\madrid.xlsx')
print('저장완료')
driver.quit()

#pane > div > div.widget-pane-content.scrollable-y > div > div > div.section-layout.section-scrollbox.scrollable-y.scrollable-show.section-layout-flex-vertical > div.section-layout.section-scrollbox.scrollable-y.scrollable-show.section-layout-flex-vertical > div:nth-child(1)
#pane > div > div.widget-pane-content.scrollable-y > div > div > div.section-layout.section-scrollbox.scrollable-y.scrollable-show.section-layout-flex-vertical > div.section-layout.section-scrollbox.scrollable-y.scrollable-show.section-layout-flex-vertical > div:nth-child(3)

In [39]:
#hdtb-msb-vis > div:nth-child(5) > a

import requests
from bs4 import BeautifulSoup
from selenium import webdriver as wd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
import os
import sys
import urllib.request
import datetime

word = "이치란"
'''
# 번역부분---------
import requests

def kor2eng(query):
    url = "https://translate.kakao.com/translator/translate.json"

    headers = {
        "Referer": "https://translate.kakao.com/",
        "User-Agent": "Mozilla/5.0"
    }

    data = {
        "queryLanguage": "ko",
        "resultLanguage": "es",
        "q": query
    }

    resp = requests.post(url, headers=headers, data=data)
    data = resp.json()
    output = data['result']['output'][0][0]
    return output


if __name__ == "__main__":
#     print(kor2eng("コロナ19に注意してください。"))
    tword=kor2eng(word)
    print(kor2eng(word))
'''    
#----------------------------

def google_searching(word):
    url = "http://www.google.com/maps/search/" + word
    return url

from selenium import webdriver


driver = webdriver.Chrome('C:/work/chromedriver')

# 구글 접속하기
driver.get('http://www.google.com/maps')
time.sleep(3)
print(1)

#검색 후 목록 보기 
url = google_searching(word)
driver.get(url)
time.sleep(3)

results = []

import re
from bs4 import BeautifulSoup


#목록에서 선택하기
def select_first(driver):
    time.sleep(3)
    try:
        #pane > div > div.widget-pane-content.scrollable-y > div > div > div.section-layout.section-scrollbox.scrollable-y.scrollable-show.section-layout-flex-vertical > div.section-layout.section-scrollbox.scrollable-y.scrollable-show.section-layout-flex-vertical > div:nth-child(1) > div.section-result-content > div.section-result-text-content > div.section-result-header-container
        first = driver.find_elements_by_css_selector("div.section-result")[0]
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
            #first = soup.select('div.section-result-title-container > h3')[2]
#         global title

        title = soup.select('div.section-result-title-container > h3')[0].text
        print(title)
        #print(first)
        first.click()
    except:
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
#         global title

        title = soup.select('h1.section-hero-header-title-title.GLOBAL__gm2-headline-5 > span')[0].text
        print(title)
        
    time.sleep(3)
select_first(driver)
print('2')

#리뷰로 넘어가기
def select_second(driver):
    driver.refresh()
    time.sleep(5)
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    #ugiz4pqJLAG__primary-text gm2-body-2
    global juso
    juso = driver.find_element_by_css_selector('div.ugiz4pqJLAG__text').text
    #juso = soup.select('div.ugiz4pqJLAG__text').text
    print(juso)
    try:
        second = driver.find_element_by_css_selector("button.widget-pane-link")
        print('s1')
        print(second.text)
        second.click()
    except:
        second = driver.find_element_by_css_selector("button.gm2-button-alt.jqnFjrOWMVU__button-blue")
        print('s2')
        print(second.text)
        second.click()
#         button.gm2-button-alt.jqnFjrOWMVU__button-blue
    
    time.sleep(3)
    #print(second)
time.sleep(3)    
select_second(driver)
print('3')

# 반복문 돌려서 리뷰 얻기.


def get_content(driver):
    #  현재 페이지 html 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')

    # 총 별점 가져오기
    tstar = soup.select('div.gm2-display-2')[0].text 
    print(tstar)

    t = 0
    s = 0
    e = 8
    while(t<20):
        time.sleep(5)
        if t == 0:
            s = 0
            e = 8
        else:
            s = 10 * (t-1) + 8
            e = s + 10
        for k in range(s, e):        
            #  리뷰 내용 가져오기
            try:
                #time.sleep(1)
                global content
                #content = soup.select('div.section-review')[t].text
#                 content = soup.select('div.section-review-review-content')[k].text
                content = driver.find_element_by_css_selector('div.section-layout.section-scrollbox.scrollable-y.scrollable-show > div:nth-child(9) > div:nth-child({}) > div > div.section-review-content > div:nth-child(3) > div.section-review-review-content'.format(3 * k + 1)).text
#                 print(content)

                #  리뷰 내용이 길 경우 자세히 버튼 눌러서 가져오기
                more = '자세히'
                if more in content:
                    more.click()
                    content = driver.find_element_by_css_selector('div.section-layout.section-scrollbox.scrollable-y.scrollable-show > div:nth-child(9) > div:nth-child({}) > div > div.section-review-content > div:nth-child(3) > div.section-review-review-content'.format(3 * k + 1)).text
                    time.sleep(3)
#                     print(content)

            except Exception as e:
                print('err : ', e)
                #content = ' '

            # 작성일 가져오기
            date = driver.find_element_by_css_selector('div.section-layout.section-scrollbox.scrollable-y.scrollable-show > div:nth-child(9) > div:nth-child({}) > div > div.section-review-content > div:nth-child(3) > div.section-review-metadata.section-review-metadata-with-note > span.section-review-publish-date'.format(3 * k + 1)).text
#             date = soup.select('span.section-review-publish-date')[k].text
#             print(date)

            # 리뷰에서 별점 가져오기
            try:
                #star = driver.find_element_by_css_selector('span.section-review-stars')
                global star
                star = int(''.join(re.findall("\d+", driver.find_element_by_css_selector('div.section-layout.section-scrollbox.scrollable-y.scrollable-show > div:nth-child(9) > div:nth-child({}) > div > div.section-review-content > div:nth-child(3) > div.section-review-metadata.section-review-metadata-with-note > span.section-review-stars'.format(3 * k + 1))
                                              .get_attribute('aria-label'))))
#                 star = soup.select('span.section-review-stars')[k]['aria-label']
                print(별)   
            except:
                print('별점 에러')


            data = [title, juso, content, date, star]
#             print(data)
            results.append(data)
        t += 1

        # 스크롤

        try:
            time.sleep(7)
            scrollable_div = driver.find_element_by_css_selector('div.section-layout.section-scrollbox.scrollable-y.scrollable-show')
            driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)
        except:
            break


get_content(driver)
    # print(results)


import pandas as pd

results_df = pd.DataFrame(results)
results_df.columns = ['title', 'juso','content','date','star']
results_df.to_excel('C:\work\madrid.xlsx')
print('저장완료')
driver.quit()

#pane > div > div.widget-pane-content.scrollable-y > div > div > div.section-layout.section-scrollbox.scrollable-y.scrollable-show.section-layout-flex-vertical > div.section-layout.section-scrollbox.scrollable-y.scrollable-show.section-layout-flex-vertical > div:nth-child(1)
#pane > div > div.widget-pane-content.scrollable-y > div > div > div.section-layout.section-scrollbox.scrollable-y.scrollable-show.section-layout-flex-vertical > div.section-layout.section-scrollbox.scrollable-y.scrollable-show.section-layout-flex-vertical > div:nth-child(3)

#pane > div > div.widget-pane-content.scrollable-y > div > div > div.jqnFjrOWMVU__root > div > div.jqnFjrOWMVU__right > button
#pane > div > div.widget-pane-content.scrollable-y > div > div > div:nth-child(45) > div > div > button

# span.reviews-tap-area.reviews-tap-area-enabled > span:nth-child(1) > button
#pane > div > div.widget-pane-content.scrollable-y > div > div > div.section-hero-header-title > div.section-hero-header-title-top-container > div.section-hero-header-title-description > div.section-hero-header-title-description-container > div > div.gm2-body-2.section-rating-line > span:nth-child(3) > span > span.section-rating-term > span:nth-child(2) > span:nth-child(1) > span
#pane > div > div.widget-pane-content.scrollable-y > div > div > div.jqnFjrOWMVU__root > div > div.jqnFjrOWMVU__right > button.gm2-button-alt.jqnFjrOWMVU__button-blue

#pane > div > div.widget-pane-content.scrollable-y > div > div > div.section-layout.section-scrollbox.scrollable-y.scrollable-show > div:nth-child(9) > div:nth-child(22) > div > div.section-review-content > div:nth-child(3) > div.section-review-metadata.section-review-metadata-with-note > span.section-review-publish-date

1
멘쇼우라멘
2
서울특별시 관악구 낙성대동 남부순환로230길 25
s1
(153)
3
4.2
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
err :  Message: no such eleme

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.section-layout.section-scrollbox.scrollable-y.scrollable-show > div:nth-child(9) > div:nth-child(460) > div > div.section-review-content > div:nth-child(3) > div.section-review-metadata.section-review-metadata-with-note > span.section-review-publish-date"}
  (Session info: chrome=87.0.4280.88)


In [57]:
51//10

5

In [8]:
#hdtb-msb-vis > div:nth-child(5) > a
def googlemapfunc(searchtext):
    import requests
    from bs4 import BeautifulSoup
    from selenium import webdriver as wd
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    import time
    import re
    import os
    import sys
    import urllib.request
    import datetime
    import pandas as pd

    word = searchtext
    '''
    # 번역부분---------
    import requests

    def kor2eng(query):
        url = "https://translate.kakao.com/translator/translate.json"

        headers = {
            "Referer": "https://translate.kakao.com/",
            "User-Agent": "Mozilla/5.0"
        }

        data = {
            "queryLanguage": "ko",
            "resultLanguage": "es",
            "q": query
        }

        resp = requests.post(url, headers=headers, data=data)
        data = resp.json()
        output = data['result']['output'][0][0]
        return output


    if __name__ == "__main__":
    #     print(kor2eng("コロナ19に注意してください。"))
        tword=kor2eng(word)
        print(kor2eng(word))
    '''    
    #----------------------------

    def google_searching(word):
        url = "http://www.google.com/maps/search/" + word
        return url

    from selenium import webdriver


    driver = webdriver.Chrome('C:/work/chromedriver')

    # 구글 접속하기
    driver.get('http://www.google.com/maps')
    time.sleep(3)
    print(1)

    #검색 후 목록 보기 
    url = google_searching(word)
    driver.get(url)
    time.sleep(3)

    results = []

    import re
    from bs4 import BeautifulSoup


    #목록에서 선택하기
    def select_first(driver,idx):
        n = 0
        time.sleep(3)
        try:
            #pane > div > div.widget-pane-content.scrollable-y > div > div > div.section-layout.section-scrollbox.scrollable-y.scrollable-show.section-layout-flex-vertical > div.section-layout.section-scrollbox.scrollable-y.scrollable-show.section-layout-flex-vertical > div:nth-child(1) > div.section-result-content > div.section-result-text-content > div.section-result-header-container
            first = driver.find_elements_by_css_selector("div.section-result")[idx]
            html = driver.page_source
            soup = BeautifulSoup(html, 'lxml')
                #first = soup.select('div.section-result-title-container > h3')[2]
    #         global title

            title = soup.select('div.section-result-title-container > h3')[0].text
            print(title)
            #print(first)
            first.click()
        except:
            if idx != 0:
                results_df = pd.DataFrame(results)
                results_df.columns = ['title', 'juso','content','date','star']
                results_df.to_excel('C:\work\{}.xlsx'.format(word))
                print('저장완료')
                driver.quit()
            html = driver.page_source
            soup = BeautifulSoup(html, 'lxml')
    #         global title

            title = soup.select('h1.section-hero-header-title-title.GLOBAL__gm2-headline-5 > span')[0].text
            print(title)

        time.sleep(3)

        print('2')

        #리뷰로 넘어가기

        driver.refresh()
        time.sleep(8)
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
        #ugiz4pqJLAG__primary-text gm2-body-2
        global juso
        juso = driver.find_element_by_css_selector('div.ugiz4pqJLAG__text').text
        #juso = soup.select('div.ugiz4pqJLAG__text').text
        print(juso)
        try:
            second = driver.find_element_by_css_selector("button.widget-pane-link")
            print('s1')
            n = int(''.join(re.findall("\d+", second.text))) // 10
            print(second.text)
            second.click()
        except:
            second = driver.find_element_by_css_selector("button.gm2-button-alt.jqnFjrOWMVU__button-blue")
            print('s2')
            n = int(''.join(re.findall("\d+", second.text))) // 10
            print(second.text)
            second.click()
        #         button.gm2-button-alt.jqnFjrOWMVU__button-blue

        time.sleep(5)
        print('3')

        # 반복문 돌려서 리뷰 얻기.



        #  현재 페이지 html 정보 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')

        # 총 별점 가져오기
        tstar = soup.select('div.gm2-display-2')[0].text 
        print(tstar)
        if n > 15:
            n = 15
        t = 0
        s = 0
        e = 8
        while(t<n):
            time.sleep(5)
            if t == 0:
                s = 0
                e = 8
            else:
                s = 10 * (t-1) + 8
                e = s + 10
            for k in range(s, e):        
                #  리뷰 내용 가져오기
                try:
                    #time.sleep(1)
                    global content
                    content = driver.find_element_by_css_selector('div.section-layout.section-scrollbox.scrollable-y.scrollable-show > div:nth-child(9) > div:nth-child({}) > div > div.section-review-content > div:nth-child(3) > div.section-review-review-content'.format(3 * k + 1)).text
        #                 print(content)

                    #  리뷰 내용이 길 경우 자세히 버튼 눌러서 가져오기
                    more = '자세히'
                    if more in content:
                        more.click()
                        content = driver.find_element_by_css_selector('div.section-layout.section-scrollbox.scrollable-y.scrollable-show > div:nth-child(9) > div:nth-child({}) > div > div.section-review-content > div:nth-child(3) > div.section-review-review-content'.format(3 * k + 1)).text
                        time.sleep(3)
        #                     print(content)
                    # 작성일 가져오기
                    date = driver.find_element_by_css_selector('div.section-layout.section-scrollbox.scrollable-y.scrollable-show > div:nth-child(9) > div:nth-child({}) > div > div.section-review-content > div:nth-child(3) > div.section-review-metadata.section-review-metadata-with-note > span.section-review-publish-date'.format(3 * k + 1)).text
            #             date = soup.select('span.section-review-publish-date')[k].text
            #             print(date)

                    # 리뷰에서 별점 가져오기
                    try:
                        #star = driver.find_element_by_css_selector('span.section-review-stars')
                        global star
                        star = int(''.join(re.findall("\d+", driver.find_element_by_css_selector('div.section-layout.section-scrollbox.scrollable-y.scrollable-show > div:nth-child(9) > div:nth-child({}) > div > div.section-review-content > div:nth-child(3) > div.section-review-metadata.section-review-metadata-with-note > span.section-review-stars'.format(3 * k + 1))
                                                      .get_attribute('aria-label'))))
            #                 star = soup.select('span.section-review-stars')[k]['aria-label']
                        print(star)   
                    except:
                        print('별점 에러')

                    data = [title, juso, content, date, star]
            #             print(data)
                    results.append(data)

                except Exception as e:
                    print('err : ', e)
                    return

            t += 1

            # 스크롤

            try:
                time.sleep(7)
                scrollable_div = driver.find_element_by_css_selector('div.section-layout.section-scrollbox.scrollable-y.scrollable-show')
                driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)
            except:
                break

    for i in range(5):
        select_first(driver, i)
        driver.refresh()
        time.sleep(5)
        back = driver.find_element_by_css_selector("button.ozj7Vb3wnYq__action-button-clickable")
        back.click()
        time.sleep(5)
        try:
            back2 = driver.find_element_by_css_selector("button.section-back-to-list-button.blue-link.noprint")
            back2.click()

        except:
            continue

    results_df = pd.DataFrame(results)
    results_df.columns = ['title', 'juso','content','date','star']
    results_df.to_excel('C:\work\{}.xlsx'.format(word))
    print('저장완료')
    driver.quit()


In [9]:
googlemapfunc('やまや')

1
야마야 이마주쿠점
2
3 Chome-40-1 Imajuku, Nishi Ward, Fukuoka, 819-0167 일본
s1
(123)
3
3.8
4
4
3
4
5
2
3
4
4
4
4
5
3
3
2
3
3
5
4
5
5
4
3
5
4
3
3
3
4
5
5
5
2
4
2
5
3
3
4
3
5
2
4
5
5
4
5
3
4
3
3
3
5
5
5
3
4
3
4
3
5
2
4
3
3
4
4
5
2
3
2
4
4
3
2
3
4
3
4
5
5
4
3
4
3
4
5
5
4
5
5
4
5
5
4
4
4
4
5
4
2
3
3
4
2
3
3
5
4
5
3
5
3
5
4
4
4
4
Yamaya
2
2 Chome-2-13 Akine Nishimachi, Shimonoseki, Yamaguchi 751-0873 일본
s1
(59)
3
3.7
5
3
4
4
4
4
3
5
3
4
5
4
4
4
4
3
5
3
3
5
3
5
5
3
4
5
4
3
3
3
4
3
3
5
5
1
5
4
5
3
5
2
5
4
3
3
4
3
Yamaya Koga
2
3 Chome-1-44 Chiko, Nogata, Fukuoka 822-0022 일본
s1
(66)
3
4.0
err :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.section-layout.section-scrollbox.scrollable-y.scrollable-show > div:nth-child(9) > div:nth-child(1) > div > div.section-review-content > div:nth-child(3) > div.section-review-review-content"}
  (Session info: chrome=87.0.4280.88)

Yamaya Koga
2
3 Chome-40-1 Imajuku, Nishi Ward, Fukuoka, 819-0167 일본
s1
(123)
3
3.8
4
4


In [48]:
#hdtb-msb-vis > div:nth-child(5) > a

import requests
from bs4 import BeautifulSoup
from selenium import webdriver as wd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
import os
import sys
import urllib.request
import datetime
import pandas as pd

word = "이치란"
'''
# 번역부분---------
import requests

def kor2eng(query):
    url = "https://translate.kakao.com/translator/translate.json"

    headers = {
        "Referer": "https://translate.kakao.com/",
        "User-Agent": "Mozilla/5.0"
    }

    data = {
        "queryLanguage": "ko",
        "resultLanguage": "es",
        "q": query
    }

    resp = requests.post(url, headers=headers, data=data)
    data = resp.json()
    output = data['result']['output'][0][0]
    return output


if __name__ == "__main__":
#     print(kor2eng("コロナ19に注意してください。"))
    tword=kor2eng(word)
    print(kor2eng(word))
'''    
#----------------------------

def google_searching(word):
    url = "http://www.google.com/maps/search/" + word
    return url

from selenium import webdriver


driver = webdriver.Chrome('C:/work/chromedriver')

# 구글 접속하기
driver.get('http://www.google.com/maps')
time.sleep(3)
print(1)

#검색 후 목록 보기 
url = google_searching(word)
driver.get(url)
time.sleep(3)

results = []
n = 0
import re
from bs4 import BeautifulSoup
time.sleep(3)
try:
    #pane > div > div.widget-pane-content.scrollable-y > div > div > div.section-layout.section-scrollbox.scrollable-y.scrollable-show.section-layout-flex-vertical > div.section-layout.section-scrollbox.scrollable-y.scrollable-show.section-layout-flex-vertical > div:nth-child(1) > div.section-result-content > div.section-result-text-content > div.section-result-header-container
    first = driver.find_elements_by_css_selector("div.section-result")[0]
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
        #first = soup.select('div.section-result-title-container > h3')[2]
#         global title

    title = soup.select('div.section-result-title-container > h3')[0].text
    print(title)
    #print(first)
    first.click()
except:
    if 0 != 0:
        results_df = pd.DataFrame(results)
        results_df.columns = ['title', 'juso','content','date','star']
        results_df.to_excel('C:\work\{}.xlsx'.format(word))
        print('저장완료')
        driver.quit()
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
#         global title

    title = soup.select('h1.section-hero-header-title-title.GLOBAL__gm2-headline-5 > span')[0].text
    print(title)

time.sleep(3)

print('2')

#리뷰로 넘어가기

driver.refresh()
time.sleep(5)
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')
#ugiz4pqJLAG__primary-text gm2-body-2
global juso
juso = driver.find_element_by_css_selector('div.ugiz4pqJLAG__text').text
#juso = soup.select('div.ugiz4pqJLAG__text').text
print(juso)
try:
    second = driver.find_element_by_css_selector("button.widget-pane-link")
    print('s1')
    print(second.text)
    second.click()
except:
    second = driver.find_element_by_css_selector("button.gm2-button-alt.jqnFjrOWMVU__button-blue")
    print('s2')
    print(second.text)
    second.click()
#         button.gm2-button-alt.jqnFjrOWMVU__button-blue

time.sleep(5)
print('3')

# 반복문 돌려서 리뷰 얻기.



#  현재 페이지 html 정보 가져오기
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

# 총 별점 가져오기
tstar = soup.select('div.gm2-display-2')[0].text 
print(tstar)

t = 0
s = 0
e = 8
while(t<11):
    time.sleep(5)
    if t == 0:
        s = 0
        e = 8
    else:
        s = 10 * (t-1) + 8
        e = s + 10
    for k in range(s, e):        
        #  리뷰 내용 가져오기
        try:
            #time.sleep(1)
            global content
            #content = soup.select('div.section-review')[t].text
#                 content = soup.select('div.section-review-review-content')[k].text
            content = driver.find_element_by_css_selector('div.section-layout.section-scrollbox.scrollable-y.scrollable-show > div:nth-child(9) > div:nth-child({}) > div > div.section-review-content > div:nth-child(3) > div.section-review-review-content'.format(3 * k + 1)).text
#                 print(content)

            #  리뷰 내용이 길 경우 자세히 버튼 눌러서 가져오기
            more = '자세히'
            if more in content:
                more.click()
                content = driver.find_element_by_css_selector('div.section-layout.section-scrollbox.scrollable-y.scrollable-show > div:nth-child(9) > div:nth-child({}) > div > div.section-review-content > div:nth-child(3) > div.section-review-review-content'.format(3 * k + 1)).text
                time.sleep(3)
#                     print(content)
            # 작성일 가져오기
            date = driver.find_element_by_css_selector('div.section-layout.section-scrollbox.scrollable-y.scrollable-show > div:nth-child(9) > div:nth-child({}) > div > div.section-review-content > div:nth-child(3) > div.section-review-metadata.section-review-metadata-with-note > span.section-review-publish-date'.format(3 * k + 1)).text
    #             date = soup.select('span.section-review-publish-date')[k].text
    #             print(date)

            # 리뷰에서 별점 가져오기
            try:
                #star = driver.find_element_by_css_selector('span.section-review-stars')
                global star
                star = int(''.join(re.findall("\d+", driver.find_element_by_css_selector('div.section-layout.section-scrollbox.scrollable-y.scrollable-show > div:nth-child(9) > div:nth-child({}) > div > div.section-review-content > div:nth-child(3) > div.section-review-metadata.section-review-metadata-with-note > span.section-review-stars'.format(3 * k + 1))
                                              .get_attribute('aria-label'))))
    #                 star = soup.select('span.section-review-stars')[k]['aria-label']
                print(별)   
            except:
                print('별점 에러')

            data = [title, juso, content, date, star]
    #             print(data)
            results.append(data)

        except Exception as e:
            print('err : ', e)
            results_df = pd.DataFrame(results)
            results_df.columns = ['title', 'juso','content','date','star']
            results_df.to_excel('C:\work\{}.xlsx'.format(word))
            print('저장완료')
            driver.quit()

    t += 1

    # 스크롤

    try:
        time.sleep(7)
        scrollable_div = driver.find_element_by_css_selector('div.section-layout.section-scrollbox.scrollable-y.scrollable-show')
        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)
    except:
        break

1
멘쇼우라멘
2
서울특별시 관악구 낙성대동 남부순환로230길 25
s1
(153)
3
4.2
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러
별점 에러


In [55]:
back = driver.find_element_by_css_selector("button.ozj7Vb3wnYq__action-button-clickable")
back.click()

In [51]:
back = driver.find_element_by_css_selector("button.section-back-to-list-button.blue-link.noprint")
back.click()


In [18]:
print(driver.find_element_by_css_selector("button.widget-pane-link").text)
driver.find_element_by_css_selector("button.widget-pane-link").click()

(1,606)


In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver as wd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
import os
import sys
import urllib.request
import datetime

word = "도쿄 맛집"
'''
# 번역부분---------
import requests

def kor2eng(query):
    url = "https://translate.kakao.com/translator/translate.json"

    headers = {
        "Referer": "https://translate.kakao.com/",
        "User-Agent": "Mozilla/5.0"
    }

    data = {
        "queryLanguage": "ko",
        "resultLanguage": "es",
        "q": query
    }

    resp = requests.post(url, headers=headers, data=data)
    data = resp.json()
    output = data['result']['output'][0][0]
    return output


if __name__ == "__main__":
#     print(kor2eng("コロナ19に注意してください。"))
    tword=kor2eng(word)
    print(kor2eng(word))
'''    
#----------------------------

def google_searching(word):
    url = "http://www.google.com/maps/search/" + word
    return url

from selenium import webdriver


driver = webdriver.Chrome('C:/work/chromedriver')

# 구글 접속하기
driver.get('http://www.google.com/maps')
time.sleep(3)
print(1)

#검색 후 목록 보기 
url = google_searching(word)
driver.get(url)
time.sleep(5)

results = []

import re
from bs4 import BeautifulSoup


#목록에서 선택하기
def select_first(driver):
    #pane > div > div.widget-pane-content.scrollable-y > div > div > div.section-layout.section-scrollbox.scrollable-y.scrollable-show.section-layout-flex-vertical > div.section-layout.section-scrollbox.scrollable-y.scrollable-show.section-layout-flex-vertical > div:nth-child(1) > div.section-result-content > div.section-result-text-content > div.section-result-header-container
    first = driver.find_element_by_css_selector("div.section-result")
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
#         #first = soup.select('div.section-result-title-container > h3')[2]
    global title
    title = soup.select('div.section-result-title-container > h3')[0].text
    print(title)
    #print(first)
    first.click()
    time.sleep(5)
select_first(driver)
print('2')

#리뷰로 넘어가기
def select_second(driver):
    second = driver.find_element_by_css_selector("span button")
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    #ugiz4pqJLAG__primary-text gm2-body-2
    global juso
    juso = driver.find_element_by_css_selector('div.ugiz4pqJLAG__text').text
    #juso = soup.select('div.ugiz4pqJLAG__text').text
    print(juso)
    second.click()
    time.sleep(5)
    #print(second)
select_second(driver)
print('3')
#pane > div > div.widget-pane-content.scrollable-y > div > div > div:nth-child(47) > div > div > button

#pane > div > div.widget-pane-content.scrollable-y > div > div > div.jqnFjrOWMVU__root > div > div.jqnFjrOWMVU__right > button

# 반복문 돌려서 리뷰 얻기.


def get_content(driver):
    #  현재 페이지 html 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    # 총 별점 가져오기
    tstar = soup.select('div.gm2-display-2')[0].text 
    print(tstar)
    
    t = 0
    s = 0
    e = 8
    while(t<200):
        time.sleep(5)
        if t == 0:
            s = 0
            e = 8
        else:
            s = 10 * (t-1) + 8
            e = s + 10
        for k in range(s, e):        
            #  리뷰 내용 가져오기
            try:
                time.sleep(3)
                global content
                #content = soup.select('div.section-review')[t].text
                content = soup.select('div.section-review-review-content')[t].text
                print(content)
            
                #  리뷰 내용이 길 경우 자세히 버튼 눌러서 가져오기
                more = '자세히'
                if more in content:
                    more.click()
                    content = soup.select('div.section-review-review-content')[t].text
                    time.sleep(3)
                    print(content)
            
            except Exception as e:
                print('err : ', e)
                #content = ' '
            
            # 작성일 가져오기
            #date = driver.find_element_by_css_selector('span.section-review-publish-date').text
            date = soup.select('span.section-review-publish-date')[1].text
            print(date)
            
            # 리뷰에서 별점 가져오기
            try:
                #star = driver.find_element_by_css_selector('span.section-review-stars')
                global star
                star = soup.select('span.section-review-stars')[1]['aria-label']
                print(별)   
            except:
                print('별점 에러')
            
#             print('div.section-layout.section-scrollbox.scrollable-y.scrollable-show > div:nth-child(9) > div:nth-child({}) > div > div.section-review-content > div.section-review-line.section-review-line-with-indent > div > button'.format(3 * k + 1))
            print(k)
#             driver.find_element_by_css_selector('div.section-layout.section-scrollbox.scrollable-y.scrollable-show > div:nth-child(9) > div:nth-child({}) > div > div.section-review-content > div.section-review-line.section-review-line-with-indent > div > button'.format(3 * k + 1)).click()
#             driver.find_element_by_css_selector('div.section-layout.section-scrollbox.scrollable-y.scrollable-show > div:nth-child(9) > div:nth-child({}) > div > div.section-review-content > div:nth-child(3) > div.section-review-interaction-bar.section-review-interaction-bar-after-photos > button:nth-child(3)'.format(3 * k + 1)).click()
#             time.sleep(2)
#             driver.find_element_by_css_selector('#modal-dialog-widget > div.modal-container > div > div.modal-close-row > button').click()
            
            #modal-dialog-widget > div.modal-container > div > div.modal-close-row > button
            data = [title, juso, content, date, star]
            print(data)
            results.append(data)
            
        scrollable_div = driver.find_element_by_css_selector(
            'div.section-layout.section-scrollbox.scrollable-y.scrollable-show'
                 )
        driver.execute_script(
           'arguments[0].scrollTop = arguments[0].scrollHeight', 
            scrollable_div
           )
        t += 1
          
        # 스크롤
        
#         time.sleep(5)
#         last_height = driver.execute_script("return document.body.scrollHeight")
#         driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#         time.sleep(5)
#         new_height = driver.execute_script("return document.body.scrollHeight")
#         if new_height == last_height:
#             driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#             time.sleep(5)
#             new_height = driver.execute_script("return document.body.scrollHeight")
#             if new_height == last_height:
#                 break

#             else:
#                 last_height = new_height
#                 continue
        
     
        
get_content(driver)
print(results)


import pandas as pd

results_df = pd.DataFrame(results)
results_df.columns = ['title', 'juso','content','date','star']
results_df.to_excel('C:\work\madrid.xlsx')
print('저장완료')
# driver.quit()

In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver as wd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
import os
import sys
import urllib.request
import datetime

In [ ]:
word = "마드리드 맛집"
def google_searching(word):
    url = "http://www.google.com/maps/search/" + word
    return url

from selenium import webdriver


driver = webdriver.Chrome('C:/work/chromedriver')

In [ ]:
# 구글 접속하기
driver.get('http://www.google.com/maps')
time.sleep(3)
print(1)

#검색 후 목록 보기 
url = google_searching(word)
driver.get(url)
time.sleep(3)

In [ ]:
results = []

import re
from bs4 import BeautifulSoup


#목록에서 선택하기
def select_first(driver):
    #pane > div > div.widget-pane-content.scrollable-y > div > div > div.section-layout.section-scrollbox.scrollable-y.scrollable-show.section-layout-flex-vertical > div.section-layout.section-scrollbox.scrollable-y.scrollable-show.section-layout-flex-vertical > div:nth-child(1) > div.section-result-content > div.section-result-text-content > div.section-result-header-container
    first = driver.find_element_by_css_selector("div.section-result")
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
#         #first = soup.select('div.section-result-title-container > h3')[2]
    global title
    title = soup.select('div.section-result-title-container > h3')[0].text
    print(title)
    #print(first)
    first.click()
    time.sleep(3)
select_first(driver)
print('2')

In [ ]:
#리뷰로 넘어가기
def select_second(driver):
    second = driver.find_element_by_css_selector("span button")
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    #ugiz4pqJLAG__primary-text gm2-body-2
    global juso
    juso = driver.find_element_by_css_selector('div.ugiz4pqJLAG__text').text
    #juso = soup.select('div.ugiz4pqJLAG__text').text
    print(juso)
    second.click()
    time.sleep(3)
    #print(second)
select_second(driver)
print('3')

In [ ]:
# 반복문 돌려서 리뷰 얻기.


def get_content(driver):
    #  현재 페이지 html 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    # 총 별점 가져오기
    tstar = soup.select('div.gm2-display-2')[0].text 
    print(tstar)
    
    t = 0
    while(t<200):
        time.sleep(5)
        for k in range(7 * t + 1, 7 * t + 8):        
            #  리뷰 내용 가져오기
            try:
                #time.sleep(1)
                global content
                #content = soup.select('div.section-review')[t].text
                content = soup.select('div.section-review-review-content')[t].text
                print(content)
            
                #  리뷰 내용이 길 경우 자세히 버튼 눌러서 가져오기
                more = '자세히'
                if more in content:
                    more.click()
                    content = soup.select('div.section-review-review-content')[t].text
                    time.sleep(3)
                    print(content)
            
            except Exception as e:
                print('err : ', e)
                #content = ' '
            
            # 작성일 가져오기
            #date = driver.find_element_by_css_selector('span.section-review-publish-date').text
            date = soup.select('span.section-review-publish-date')[t].text
            print(date)
            
            # 리뷰에서 별점 가져오기
            try:
                #star = driver.find_element_by_css_selector('span.section-review-stars')
                global star
                star = soup.select('span.section-review-stars')[t]['aria-label']
                print(별)   
            except:
                print('별점 에러')
                
            
            data = [title, juso, content, date, star]
            print(data)
            results.append(data)
            t += 1
          
        # 스크롤
        
        time.sleep(5)
        last_height = driver.execute_script("return document.body.scrollHeight")
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break

            else:
                last_height = new_height
                continue
        
     
        
get_content(driver)
print(results)

In [ ]:
import pandas as pd

results_df = pd.DataFrame(results)
results_df.columns = ['title', 'juso','content','date','star']
results_df.to_excel('C:\work\madrid.xlsx')
print('저장완료')

In [ ]:
while(True):
    last_height = driver.execute_script("return document.body.scrollHeight")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break

        else:
            last_height = new_height
            continue


In [ ]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
#pane > div > div.widget-pane-content.scrollable-y > div > div > div.section-layout.section-scrollbox.scrollable-y.scrollable-show

#pane > div > div.widget-pane-content.scrollable-y > div > div > div.section-layout.section-scrollbox.scrollable-y.scrollable-show > div:nth-child(9) > div:nth-child(1) > div > div.section-review-content > div.section-review-line.section-review-line-with-indent > div > button > img
#pane > div > div.widget-pane-content.scrollable-y > div > div > div.section-layout.section-scrollbox.scrollable-y.scrollable-show > div:nth-child(9) > div:nth-child(4) > div > div.section-review-content > div.section-review-line.section-review-line-with-indent > div > button > img
#pane > div > div.widget-pane-content.scrollable-y > div > div > div.section-layout.section-scrollbox.scrollable-y.scrollable-show > div:nth-child(9) > div:nth-child(7) > div > div.section-review-content > div.section-review-line.section-review-line-with-indent > div > button > img
#pane > div > div.widget-pane-content.scrollable-y.scrollable-show > div > div > div.section-layout.section-scrollbox.scrollable-y.scrollable-show > div:nth-child(9) > div:nth-child(85) > div > div.section-review-content > div.section-review-line.section-review-line-with-indent > div > button > img

In [ ]:
from selenium.webdriver.common.keys import Keys
body = driver.find_element_by_css_selector('div')
for i in range(10):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)